# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Student 1 - __[student1@ulb.ac.be](mailto:student1@ulb.ac.be) - Student ID 123456__
### Student 2 - __[student2@vub.ac.be](mailto:student2@ulb.ac.be) - Student ID 789012__
### Student 3 - __[student3@ulb.ac.be](mailto:student3@ulb.ac.be) - Student ID 345678__

### Video presentation: www.youtube.com/abcd1234

## Project Title


# Introduction


In [1]:
#install.packages("e1071")
#install.packages("tidyverse")
#install.packages("pillar")
#install.packages("caret")

#install.packages("drat", repos="https://cran.rstudio.com")
#drat:::addRepo("dmlc")
#install.packages("xgboost", repos="http://dmlc.ml/drat/", type = "source")

#install.packages("FactoMineR")
#install.packages("factoextra")

In [2]:
library(rpart)
library(e1071)
library(rpart.plot)
library(randomForest)
library(nnet)
library(xgboost)
library(caret)
library(FactoMineR)
library(factoextra)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [3]:
sub_format <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Submission_format.csv", header=TRUE)
set_val_test <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Test_set_values.csv", header=TRUE)
set_labels <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv", header=TRUE)
dataset <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv", header=TRUE)

In [4]:
choices <- table(set_labels$status_group)
choices

[1] 3

In [5]:
dim(set_labels)

#apply(is.na(set_val),2,any)

[1] 59400     2

In [6]:
dataset$funder[0:40]#Problem mac, don't have na values

[1] "Roman"                    "Grumeti"                 
 [3] "Lottery Club"             "Unicef"                  
 [5] "Action In A"              "Mkinga Distric Coun"     
 [7] "Dwsp"                     "Rwssp"                   
 [9] "Wateraid"                 "Isingiro Ho"             
[11] "Private"                  "Danida"                  
[13] "World Vision"             "Lawatefuka Water Supply" 
[15] "Biore"                    "Rudep"                   
[17] "Unicef"                   "Unicef"                  
[19] "Hesawa"                   "Danida"                  
[21] "Twe"                      "Dwsp"                    
[23] "Unicef"                   "Isf"                     
[25] "African Development Bank" "Government Of Tanzania"  
[27] "Sobodo"                   "Hesawa"                  
[29] "Government Of Tanzania"   "Lawatefuka Water Supply" 
[31] "Water"                    "Private Individual"      
[33] "Government Of Tanzania"   "Undp"                    
[35] ""                         "Not Known"               
[37] "Danida"                   "Kirde"                   
[39] "Cefa"                     "Government Of Tanzania"

In [7]:
data1<-replace(dataset,dataset=='',NA)#Replaced to NA
#data1[20:40]
dim(na.omit(data1))

[1] 27813    40

'''date_recorded
num_private
funder
installer
wpt_name
subvillage
region? (code 27, region 21, why?)
lga
ward
recorded_by
scheme_name
payment
quantity
waterpoint_type_group'''

In [8]:
cleaning <- c("date_recorded", "num_private", "funder", "installer", "wpt_name", "subvillage", "lga", "ward", 
              "region_code", "scheme_name", "payment", "quantity", "waterpoint_type_group", "recorded_by")
X<-dataset[,setdiff(colnames(dataset),cleaning)]
X<-replace(X,X=='',NA)#Replaced to NA
#X[20:26]

In [9]:
choices <- length(unique(X$construction_year))
choices
choices2 <- unique(X$construction_year)
choices2

[1] 55

[1] 1999 2010 2009 1986    0 2011 1987 1991 1978 1992 2008 1974 2000 2002 2004
[16] 1972 2003 1980 2007 1973 1985 1970 1995 2006 1962 2005 1997 2012 1996 1977
[31] 1983 1984 1990 1982 1976 1988 1989 1975 1960 1961 1998 1963 1971 1994 1968
[46] 1993 2001 1979 1967 2013 1969 1981 1964 1966 1965

In [10]:
#distric code 0 ?? a demander
#population 0 -> mean ???
#public_meeting -> plus frequent
#scheme_management -> frequent
#permit -> plus frequent
#construction_year -> int(mean)

#dim(data.frame(X$permit))

replace_na_with_mean_value<-function(vec) {
    mean_vec<-mean(vec,na.rm=T)
    vec[is.na(vec)]<-mean_vec
    vec
}

table(X$permit)
X$permit[is.na(X$permit)] <- tail(names(sort(table(X$permit))), 1)
table(X$permit)

table(X$scheme_management)
X$scheme_management[is.na(X$scheme_management)] <- tail(names(sort(table(X$scheme_management))), 1)
table(X$scheme_management)

table(X$public_meeting)
X$public_meeting[is.na(X$public_meeting)] <- tail(names(sort(table(X$public_meeting))), 1)
table(X$public_meeting)


False  True 
17492 38852 


False  True 
17492 41908 


         Company             None            Other       Parastatal 
            1061                1              766             1680 
Private operator              SWC            Trust              VWC 
            1063               97               72            36793 
 Water authority      Water Board              WUA              WUG 
            3153             2748             2883             5206 


         Company             None            Other       Parastatal 
            1061                1              766             1680 
Private operator              SWC            Trust              VWC 
            1063               97               72            40670 
 Water authority      Water Board              WUA              WUG 
            3153             2748             2883             5206 


False  True 
 5055 51011 


False  True 
 5055 54345 

In [11]:
X

id,amount_tsh,gps_height,longitude,latitude,basin,region,district_code,population,public_meeting,⋯,management,management_group,payment_type,water_quality,quality_group,quantity_group,source,source_type,source_class,waterpoint_type
<int>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
69572,6000,1390,34.93809,-9.85632177,Lake Nyasa,Iringa,5,109,True,⋯,vwc,user-group,annually,soft,good,enough,spring,spring,groundwater,communal standpipe
8776,0,1399,34.69877,-2.14746569,Lake Victoria,Mara,2,280,True,⋯,wug,user-group,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe
34310,25,686,37.46066,-3.82132853,Pangani,Manyara,4,250,True,⋯,vwc,user-group,per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple
67743,0,263,38.48616,-11.15529772,Ruvuma / Southern Coast,Mtwara,63,58,True,⋯,vwc,user-group,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple
19728,0,0,31.13085,-1.82535885,Lake Victoria,Kagera,1,0,True,⋯,other,other,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe
9944,20,0,39.17280,-4.76558728,Pangani,Tanga,8,1,True,⋯,vwc,user-group,per bucket,salty,salty,enough,other,other,unknown,communal standpipe multiple
19816,0,0,33.36241,-3.76636472,Internal,Shinyanga,3,0,True,⋯,vwc,user-group,never pay,soft,good,enough,machine dbh,borehole,groundwater,hand pump
54551,0,0,32.62062,-4.22619802,Lake Tanganyika,Shinyanga,3,0,True,⋯,wug,user-group,unknown,milky,milky,enough,shallow well,shallow well,groundwater,hand pump
53934,0,0,32.71110,-5.14671181,Lake Tanganyika,Tabora,6,0,True,⋯,vwc,user-group,never pay,salty,salty,seasonal,machine dbh,borehole,groundwater,hand pump


In [12]:
X$construction_year[X$construction_year==0] <- NA
X$construction_year[is.na(X$construction_year)] <- mean(X$construction_year,na.rm=T)
X$construction_year <- as.integer(X$construction_year)

In [33]:
X$longitude[X$longitude==0] <- NA
X$longitude[is.na(X$longitude)] <- mean(X$longitude,na.rm=T)

#X$latitude[X$latitude==0] <- NA
#X$latitude[is.na(X$latitude)] <- mean(X$latitude,na.rm=T)
sum(X$latitude==-0.00000002)
table(X$latitude)

[1] 1812


-11.64944018 -11.64837759 -11.58629656 -11.56857679 -11.56680457 -11.56450865 
           1            1            1            1            1            1 
-11.56432357 -11.56231592 -11.56228898 -11.56161898 -11.56105111 -11.56054536 
           1            1            1            1            1            1 
-11.55748998 -11.55727601 -11.55703116  -11.5563898 -11.55413006 -11.55316758 
           1            1            1            1            1            1 
-11.53952862   -11.538421 -11.53833213 -11.53785015 -11.53706812 -11.53678512 
           1            1            1            1            1            1 
-11.52131166 -11.51493668 -11.51388585 -11.51323073 -11.51175844 -11.51079105 
           1            1            1            1            1            1 
-11.50940228 -11.50923897 -11.50897395 -11.50843874 -11.50657735 -11.50565485 
           1            1            1            1            1            1 
-11.50395039 -11.50120749  -11.4980626 -11.49638918

In [14]:
X$permit[X$permit=='True'] <- 1
X$permit[X$permit=='False'] <- 0

X$permit <- as.integer(X$permit)

X$public_meeting[X$public_meeting=='True'] <- 1
X$public_meeting[X$public_meeting=='False'] <- 0

X$public_meeting <- as.integer(X$public_meeting)

In [15]:
summary(X)

       id          amount_tsh         gps_height       longitude    
 Min.   :    0   Min.   :     0.0   Min.   : -90.0   Min.   :29.61  
 1st Qu.:18520   1st Qu.:     0.0   1st Qu.:   0.0   1st Qu.:33.35  
 Median :37062   Median :     0.0   Median : 369.0   Median :35.15  
 Mean   :37115   Mean   :   317.7   Mean   : 668.3   Mean   :35.15  
 3rd Qu.:55656   3rd Qu.:    20.0   3rd Qu.:1319.2   3rd Qu.:37.18  
 Max.   :74247   Max.   :350000.0   Max.   :2770.0   Max.   :40.35  
    latitude          basin              region          district_code  
 Min.   :-11.649   Length:59400       Length:59400       Min.   : 0.00  
 1st Qu.: -8.541   Class :character   Class :character   1st Qu.: 2.00  
 Median : -5.022   Mode  :character   Mode  :character   Median : 3.00  
 Mean   : -5.706                                         Mean   : 5.63  
 3rd Qu.: -3.326                                         3rd Qu.: 5.00  
 Max.   :  0.000                                         Max.   :80.00  
   pop

In [16]:
names(X)

[1] "id"                    "amount_tsh"            "gps_height"           
 [4] "longitude"             "latitude"              "basin"                
 [7] "region"                "district_code"         "population"           
[10] "public_meeting"        "scheme_management"     "permit"               
[13] "construction_year"     "extraction_type"       "extraction_type_group"
[16] "extraction_type_class" "management"            "management_group"     
[19] "payment_type"          "water_quality"         "quality_group"        
[22] "quantity_group"        "source"                "source_type"          
[25] "source_class"          "waterpoint_type"

In [17]:
#one hot converstion
#install.packages('dummies')
library(dummies)


variables_to_keep<-names(X)
X_onehot <- dummy.data.frame(X[,variables_to_keep], sep="_")

dummies-1.5.6 provided by Decision Patterns


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 

In [18]:
#names(data_factor_onehot)
X_onehot

,id,amount_tsh,gps_height,longitude,latitude,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,⋯,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,69572,6000,1390,34.93809,-9.85632177,0,1,0,0,0,⋯,1,0,0,0,1,0,0,0,0,0
2,8776,0,1399,34.69877,-2.14746569,0,0,0,0,1,⋯,0,1,0,0,1,0,0,0,0,0
3,34310,25,686,37.46066,-3.82132853,0,0,0,0,0,⋯,0,1,0,0,0,1,0,0,0,0
4,67743,0,263,38.48616,-11.15529772,0,0,0,0,0,⋯,1,0,0,0,0,1,0,0,0,0
5,19728,0,0,31.13085,-1.82535885,0,0,0,0,1,⋯,0,1,0,0,1,0,0,0,0,0
6,9944,20,0,39.17280,-4.76558728,0,0,0,0,0,⋯,0,0,1,0,0,1,0,0,0,0
7,19816,0,0,33.36241,-3.76636472,1,0,0,0,0,⋯,1,0,0,0,0,0,0,1,0,0
8,54551,0,0,32.62062,-4.22619802,0,0,0,1,0,⋯,1,0,0,0,0,0,0,1,0,0
9,53934,0,0,32.71110,-5.14671181,0,0,0,1,0,⋯,1,0,0,0,0,0,0,1,0,0


In [19]:
vars <- c("id", "status_group")
set_labels <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv", header=TRUE)

labels_onehot <- dummy.data.frame(set_labels[,vars], sep="_")

names(labels_onehot)[names(labels_onehot) == "status_group_functional"] <- "functional"
names(labels_onehot)[names(labels_onehot) == "status_group_functional needs repair"] <- "functional_needs_repair"
names(labels_onehot)[names(labels_onehot) == "status_group_non functional"] <- "non_functional"
labels_onehot

#set_labels$status_group[set_labels$status_group=='functional']<-0
#set_labels$status_group[set_labels$status_group=='functional needs repair']<-0.5
#set_labels$status_group[set_labels$status_group=='non functional']<-1

#set_labels$status_group <- as.double(set_labels$status_group)
#set_labels


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”


,id,functional,functional_needs_repair,non_functional
,<int>,<int>,<int>,<int>
1,69572,1,0,0
2,8776,1,0,0
3,34310,1,0,0
4,67743,0,0,1
5,19728,1,0,0
6,9944,1,0,0
7,19816,0,0,1
8,54551,0,0,1
9,53934,0,0,1


In [20]:
Y<-labels_onehot[,setdiff(colnames(labels_onehot),"id")]
Y<-set_labels[2]
Y[Y == "functional needs repair"] <- "functional_needs_repair"
Y[Y == "non functional"] <- "non_functional"
Y

status_group
<chr>
functional
functional
functional
non_functional
functional
functional
non_functional
non_functional
non_functional


In [30]:
X_final<-X_onehot[,setdiff(colnames(X_onehot),"id")]
X_final

,amount_tsh,gps_height,longitude,latitude,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,⋯,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,6000,1390,34.93809,-9.85632177,0,1,0,0,0,0,⋯,1,0,0,0,1,0,0,0,0,0
2,0,1399,34.69877,-2.14746569,0,0,0,0,1,0,⋯,0,1,0,0,1,0,0,0,0,0
3,25,686,37.46066,-3.82132853,0,0,0,0,0,1,⋯,0,1,0,0,0,1,0,0,0,0
4,0,263,38.48616,-11.15529772,0,0,0,0,0,0,⋯,1,0,0,0,0,1,0,0,0,0
5,0,0,31.13085,-1.82535885,0,0,0,0,1,0,⋯,0,1,0,0,1,0,0,0,0,0
6,20,0,39.17280,-4.76558728,0,0,0,0,0,1,⋯,0,0,1,0,0,1,0,0,0,0
7,0,0,33.36241,-3.76636472,1,0,0,0,0,0,⋯,1,0,0,0,0,0,0,1,0,0
8,0,0,32.62062,-4.22619802,0,0,0,1,0,0,⋯,1,0,0,0,0,0,0,1,0,0
9,0,0,32.71110,-5.14671181,0,0,0,1,0,0,⋯,1,0,0,0,0,0,0,1,0,0


In [22]:
N<-nrow(X_final)    #Number of examples
n<-ncol(X_final)    #Number of input variables

names(X_final)<-make.names(names(X_final))

# PCA

In [23]:
#X_pca<-data.frame(prcomp(X_final,retx=T)$x)

X_pca <- prcomp(X_final, scale = TRUE)#,retx=T

In [24]:
s <- summary(X_pca)
s$importance[3,]

List of 6
 $ sdev      : num [1:159] 3.24 2.63 2.37 2.22 2.12 ...
 $ rotation  : num [1:159, 1:159] 0.0205 0.1332 0.0229 -0.0527 -0.0253 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:159] "amount_tsh" "gps_height" "longitude" "latitude" ...
  .. ..$ : chr [1:159] "PC1" "PC2" "PC3" "PC4" ...
 $ center    : Named num [1:159] 317.65 668.297 35.15 -5.706 0.131 ...
  ..- attr(*, "names")= chr [1:159] "amount_tsh" "gps_height" "longitude" "latitude" ...
 $ scale     : Named num [1:159] 2997.575 693.116 2.567 2.946 0.337 ...
  ..- attr(*, "names")= chr [1:159] "amount_tsh" "gps_height" "longitude" "latitude" ...
 $ x         : num [1:59400, 1:159] 3.55 2.03 2.66 -1.28 2.37 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:59400] "1" "2" "3" "4" ...
  .. ..$ : chr [1:159] "PC1" "PC2" "PC3" "PC4" ...
 $ importance: num [1:3, 1:159] 3.2392 0.066 0.066 2.6252 0.0433 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:3] "Standard deviation" "Proportion of Variance" "Cumu

PC1     PC2     PC3     PC4     PC5     PC6     PC7     PC8     PC9    PC10 
0.06599 0.10933 0.14457 0.17567 0.20391 0.22896 0.25171 0.27379 0.29502 0.31527 
   PC11    PC12    PC13    PC14    PC15    PC16    PC17    PC18    PC19    PC20 
0.33485 0.35399 0.37302 0.39128 0.40909 0.42637 0.44276 0.45864 0.47342 0.48793 
   PC21    PC22    PC23    PC24    PC25    PC26    PC27    PC28    PC29    PC30 
0.50201 0.51557 0.52905 0.54238 0.55539 0.56833 0.58091 0.59330 0.60540 0.61731 
   PC31    PC32    PC33    PC34    PC35    PC36    PC37    PC38    PC39    PC40 
0.62900 0.64068 0.65189 0.66285 0.67358 0.68390 0.69399 0.70333 0.71257 0.72152 
   PC41    PC42    PC43    PC44    PC45    PC46    PC47    PC48    PC49    PC50 
0.72994 0.73826 0.74619 0.75396 0.76138 0.76872 0.77578 0.78271 0.78954 0.79621 
   PC51    PC52    PC53    PC54    PC55    PC56    PC57    PC58    PC59    PC60 
0.80273 0.80917 0.81547 0.82177 0.82806 0.83431 0.84055 0.84675 0.85289 0.85897 
   PC61    PC62    PC63    PC64    PC65    PC66    PC67    PC68    PC69    PC70 
0.86489 0.87078 0.87657 0.88231 0.88789 0.89338 0.89878 0.90408 0.90925 0.91430 
   PC71    PC72    PC73    PC74    PC75    PC76    PC77    PC78    PC79    PC80 
0.91921 0.92405 0.92867 0.93315 0.93750 0.94169 0.94577 0.94961 0.95338 0.95707 
   PC81    PC82    PC83    PC84    PC85    PC86    PC87    PC88    PC89    PC90 
0.96062 0.96411 0.96724 0.97017 0.97287 0.97550 0.97802 0.98036 0.98258 0.98455 
   PC91    PC92    PC93    PC94    PC95    PC96    PC97    PC98    PC99   PC100 
0.98635 0.98792 0.98942 0.99071 0.99192 0.99311 0.99422 0.99519 0.99599 0.99673 
  PC101   PC102   PC103   PC104   PC105   PC106   PC107   PC108   PC109   PC110 
0.99744 0.99808 0.99864 0.99909 0.99949 0.99972 0.99994 1.00000 1.00000 1.00000 
  PC111   PC112   PC113   PC114   PC115   PC116   PC117   PC118   PC119   PC120 
1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 
  PC121   PC122   PC123   PC124   PC125   PC126   PC127   PC128   PC129   PC130 
1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 
  PC131   PC132   PC133   PC134   PC135   PC136   PC137   PC138   PC139   PC140 
1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 
  PC141   PC142   PC143   PC144   PC145   PC146   PC147   PC148   PC149   PC150 
1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 
  PC151   PC152   PC153   PC154   PC155   PC156   PC157   PC158   PC159 
1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000 1.00000

In [25]:
X_final <- data.frame(X_pca$x[,1:68])
X_final

#FOR NEW DATA HENCE FINAL SUBMISSION: PCA_testing = predict(X_pca, X_test)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,⋯,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3.545264,1.69335666,2.88692929,0.175382461,-0.038367367,-0.5384677,-1.51762427,0.337475762,-0.33357712,1.0632189,⋯,0.38773767,-0.175604727,-1.2399517,-0.41949019,0.42342859,0.028382372,-0.070680350,0.40964095,-0.569938603,-0.63420379
2,2.031601,1.09164064,-5.62783302,-0.492572219,0.704656354,-2.4613248,-1.37852481,1.326733683,1.19092904,-1.8649848,⋯,0.10961245,0.005779238,-1.5016386,0.10328583,-0.01864019,0.484838311,1.187152330,2.25338763,-1.140825998,2.38236856
3,2.659500,-1.44632296,-0.93486474,-0.093152283,-0.847906097,0.3143014,1.99031127,-0.652741906,1.43984271,-3.2041434,⋯,-1.09360830,-0.937961914,-0.4399054,0.98445328,-0.85026184,0.331273394,-0.749979849,-0.15876131,1.728009607,-1.64281572
4,-1.281717,-5.46121396,3.16137515,-0.104246764,-1.323667571,0.6032480,-2.22107880,-2.644511414,-0.98902386,-1.7716829,⋯,0.47016728,1.280669438,-0.8937745,0.24348927,-0.98299924,0.008826275,1.146707419,-0.38943447,0.246428187,0.33025198
5,2.372133,0.94715996,-6.69233969,-1.685463085,-0.158276420,-4.0712593,-3.19615484,0.816533136,2.15706884,-2.7636039,⋯,1.50101391,0.779290629,0.4876831,1.00273315,0.26768819,-0.615941853,-0.974376356,-0.36167487,1.019180985,-0.38465722
6,-1.426238,-5.94900023,0.95410697,-3.367081856,1.540051672,6.0454608,0.87717800,7.074446783,-7.00974577,-8.0503386,⋯,-0.16441324,2.077768633,0.3428631,-0.34622694,0.43546039,-0.215473297,0.552911575,-0.12702459,0.388591926,-0.93836277
7,-3.010711,0.48483628,1.00347761,2.076179926,-0.227196848,-1.6240085,-0.09754555,-0.487284713,-0.26442181,-0.5548768,⋯,-0.07287279,-0.197427556,-0.5842096,-0.29080488,-0.21505775,0.063902915,0.474555790,-0.62968638,-0.036504235,0.22928623
8,-5.937812,3.28344688,-1.78955840,-0.255488583,0.793642108,1.5780977,3.94151292,0.956296754,-1.26436776,1.6336476,⋯,0.36803571,-0.336204902,-0.2706909,-0.03583553,-0.36898936,-0.286036869,0.383304248,-0.10332489,0.276724821,-0.26036515
9,-4.478322,-1.31522764,1.56385792,0.066325431,-0.795653427,-1.9091563,-0.67196674,2.984995350,-0.55019927,-3.9389686,⋯,-0.39870051,0.184366756,0.4207338,-1.49143317,1.24526778,-0.882468947,-1.498915417,-1.11223550,0.038075273,1.05840399


In [26]:
#Another way to get the same eigen values

library(FactoMineR)
library(factoextra)

In [31]:
ev <- X_pca$sdev^2
#ev
res.pca <- PCA(X_final, graph = FALSE)
res.pca$eig

,eigenvalue,percentage of variance,cumulative percentage of variance
comp 1,10.492146,6.598834,6.598834
comp 2,6.891488,4.334269,10.933103
comp 3,5.602939,3.523861,14.456965
comp 4,4.944499,3.109748,17.566712
comp 5,4.490790,2.824396,20.391109
comp 6,3.982367,2.504633,22.895742
comp 7,3.616884,2.274770,25.170512
comp 8,3.512210,2.208937,27.379449
comp 9,3.374147,2.122105,29.501554
comp 10,3.221225,2.025928,31.527482


In [32]:
res.pca$var

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
amount_tsh,-0.06652551,0.010274745,-0.0114579618,-0.0528476599,-0.056342277
gps_height,-0.43155629,-0.108326183,-0.0895155317,0.0074241702,0.132458794
longitude,-0.07415140,0.457516449,-0.2051201789,-0.0285805094,0.159673270
latitude,0.17063303,-0.175676893,0.3437591779,0.0565342306,0.460958046
basin_Internal,0.08199169,0.091496479,0.1131027786,0.0433841142,0.092657734
basin_Lake Nyasa,-0.24044717,-0.166700639,-0.1962798961,0.0097404332,-0.078596442
basin_Lake Rukwa,0.04756272,-0.069078789,-0.0534595847,-0.0585509955,-0.190529322
basin_Lake Tanganyika,0.07304528,-0.147071557,0.1717707498,0.0741869289,-0.089670797
basin_Lake Victoria,0.30847414,-0.246599383,0.2274938409,0.0135819343,0.118602452
basin_Pangani,-0.32651455,0.011435198,-0.1039702486,0.0621151830,0.510653578


In [25]:
#DS<-cbind(X_final,label=Y)
#DS<-cbind(X_pca,label=Y)
#names(DS)<-make.names(names(DS))
#DS$status_group<-factor(DS$status_group)

In [26]:
#DS

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,⋯,PC151,PC152,PC153,PC154,PC155,PC156,PC157,PC158,PC159,status_group
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-5694.65598,592.626326,-186.203084,2.3411072,-1.6078650,3.1386680,-2.14335613,0.496976205,1.098876923,-0.3718124,⋯,8.399314e-16,1.154675e-16,-1.094079e-16,2.982721e-18,-1.266122e-16,-8.708282e-17,-8.376868e-17,-3.070447e-17,1.978124e-17,functional
2,303.63567,743.109148,-21.352283,13.0292677,3.3197694,-3.2369737,1.31350858,1.012054200,-0.650420512,0.8549088,⋯,-2.835189e-16,3.485710e-16,1.652578e-16,1.478584e-16,-2.378166e-16,-4.537661e-17,-2.264111e-16,8.053468e-17,-1.187746e-17,functional
3,292.07535,34.510625,66.124405,12.2689017,2.6605354,-0.2701010,2.79118688,1.182500162,-0.321827684,0.2028049,⋯,3.894454e-16,-3.745919e-16,-1.344292e-16,2.071066e-16,-2.207006e-16,5.136620e-18,6.695956e-17,-1.478895e-17,3.631927e-17,functional
4,325.52024,-413.856576,-53.723258,-5.8008588,-57.5539431,0.8034112,-1.13208940,-0.476679276,-0.847686803,-1.1755996,⋯,-5.965280e-16,-1.441989e-17,-6.388492e-17,-4.721245e-17,5.701019e-17,2.254550e-16,1.916314e-16,-1.778527e-18,-1.442139e-17,non_functional
5,330.59903,-682.621009,-67.889513,-0.7626552,6.5200030,-4.5858135,-1.54205853,2.067721343,-0.564478420,0.2155048,⋯,-2.835189e-16,7.101524e-17,-1.018896e-16,2.369798e-16,5.405065e-17,1.346823e-16,7.066025e-17,-9.684080e-17,2.411805e-17,functional
6,310.59983,-682.846965,-66.865744,12.8741376,-0.2025347,1.2369008,3.91634214,0.002843854,-0.819028696,-0.9050292,⋯,9.194034e-16,-1.704474e-15,-1.560997e-16,-3.313731e-17,1.667111e-16,-1.985888e-16,4.638252e-16,-1.446369e-17,-1.141680e-16,functional
7,330.59909,-682.626383,-67.883428,-0.5602538,4.2262372,-2.0422393,-0.57065274,-0.738957002,-0.088323096,-0.8394809,⋯,4.266336e-16,-1.861575e-16,-5.765076e-17,-1.125463e-17,-5.507472e-17,-5.215161e-17,1.270205e-16,1.997836e-17,1.938762e-17,non_functional
8,330.59909,-682.626880,-67.883551,-0.5672506,4.2629810,-2.1878232,-1.50653340,-1.437341891,0.205704523,0.7416535,⋯,7.943949e-16,-4.845300e-16,8.692760e-17,1.180229e-16,2.282521e-17,6.575538e-17,3.959317e-17,-1.094754e-16,-7.934275e-18,non_functional
9,330.59927,-682.633044,-67.876729,-0.3394667,1.1867450,-1.4623933,-1.89990028,-0.988586346,-0.286550443,-1.2895820,⋯,-8.154285e-16,3.238512e-16,-1.428267e-17,-3.648130e-16,1.294565e-16,-6.049996e-17,-3.342533e-16,3.721717e-17,-1.183740e-17,non_functional


In [56]:
model<- rpart(status_group~.,data=DS, method = "class") ### Fill with your code here

In [59]:
Y_hat_ts<- predict(model,X_ts)
Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))

Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
Y_hat <- ifelse(Y_hat[,1] > threshold,"functional",ifelse(Y_hat[,2] > threshold,"functional_needs_repair","non_functional"))
acc <- confusionMatrix(as.factor(Y_hat),as.factor(Y_ts))$overall['Accuracy']
acc

Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(Y_ts)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


Accuracy 
0.7128788

# nnet

In [81]:
#test without cross validation

CV_folds <- 10

size_CV <-floor(N/CV_folds)
    
idx_ts<-(((1-1)*size_CV+1):(1*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
X_ts<-X_final[idx_ts,]  
Y_ts<-Y[idx_ts,]
Y_onehot<-labels_onehot[idx_ts,]

idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
X_tr<-X_final[idx_tr,]
Y_tr<-Y[idx_tr,]         

DS<-cbind(X_tr,status_group=Y_tr)
DS$status_group<-factor(DS$status_group)
    
#model<- nnet(status_group~.,data=DS, size=10, maxit=1000, trace=T, MaxNWts=2000)#1000 max iterations ?
#Y_hat_ts<- predict(model,X_ts)

In [26]:
function(x){
   (x-min(x))/(max(x)-min(x))
}

function(x){
   (x-min(x))/(max(x)-min(x))
}

In [ ]:
#Y_onehot<-Y_onehot[,setdiff(colnames(Y_onehot),"id")]
#threshold <- 0.5
#Y_hat <- ifelse(Y_hat_ts[,1] > threshold,"functional",ifelse(Y_hat_ts[,2] > threshold,"functional_needs_repair","non_functional"))
#data.matrix(Y_hat)

In [ ]:
#table(data.frame(Y_hat),data.frame(Y))

#require(caret)
#acc <- confusionMatrix(as.factor(Y_hat),as.factor(Y[idx_ts,]))$overall['Accuracy']
#confusionMatrix(as.factor(Y_hat),as.factor(Y[idx_ts,]))

In [ ]:
#table(Y)

In [83]:
#nnet model cross validation

CV_folds <- 5

size_CV <-floor(N/CV_folds)

CV_err<-matrix(0,nrow=1,ncol=CV_folds)
threshold <- 0.5


for (i in 1:CV_folds) {
    
    idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
    X_ts<-X_final[idx_ts,]  
    Y_ts<-Y[idx_ts,]
     
    idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
    X_tr<-X_final[idx_tr,]
    Y_tr<-Y[idx_tr,]         
        # Create a dataset including only the first nb_components principal components
    DS<-cbind(X_tr,status_group=Y_tr)
    DS$status_group<-factor(DS$status_group)

    # Model fit (using lm function)
    model<- nnet(status_group~.,data=DS, size=10, decay=1.0e-5, maxit=1000, MaxNWts=2000,trace=T)#1000 max iterations ?

    # Model predict
    Y_hat_ts<- predict(model,X_ts)
    Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))
                 
    Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
    Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
    Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
    Y_hat <- ifelse(Y_hat[,1] > threshold,"functional",ifelse(Y_hat[,2] > threshold,"functional_needs_repair","non_functional"))
    acc <- confusionMatrix(as.factor(Y_hat),as.factor(Y_ts))$overall['Accuracy']
    print(acc)
} 

# weights:  773
initial  value 54535.726204 
iter  10 value 34024.414403
iter  20 value 30762.421710
iter  30 value 29462.801723
iter  40 value 28970.526621
iter  50 value 28712.635775
iter  60 value 28535.091313
iter  70 value 28376.998439
iter  80 value 28223.821450
iter  90 value 28088.004206
iter 100 value 27982.437987
iter 110 value 27906.762812
iter 120 value 27848.824540
iter 130 value 27818.515233
iter 140 value 27786.524365
iter 150 value 27755.681017
iter 160 value 27729.128037
iter 170 value 27708.276752
iter 180 value 27683.187967
iter 190 value 27649.954100
iter 200 value 27612.140106
iter 210 value 27578.499752
iter 220 value 27534.728255
iter 230 value 27489.954824
iter 240 value 27451.880812
iter 250 value 27420.729929
iter 260 value 27393.313820
iter 270 value 27359.575123
iter 280 value 27322.841499
iter 290 value 27280.417969
iter 300 value 27243.273429
iter 310 value 27212.232633
iter 320 value 27175.689664
iter 330 value 27139.854512
iter 340 value 27104.468308
ite

initial  value 65385.546268 
iter  10 value 35621.450180
iter  20 value 31684.763240
iter  30 value 30165.182194
iter  40 value 29272.498214
iter  50 value 28949.220341
iter  60 value 28775.895655
iter  70 value 28600.334317
iter  80 value 28390.477603
iter  90 value 28228.202149
iter 100 value 28064.863944
iter 110 value 27948.027564
iter 120 value 27843.766203
iter 130 value 27756.969504
iter 140 value 27681.400619
iter 150 value 27627.452501
iter 160 value 27585.896534
iter 170 value 27561.031575
iter 180 value 27539.043291
iter 190 value 27517.976613
iter 200 value 27490.764212
iter 210 value 27468.235836
iter 220 value 27442.997139
iter 230 value 27418.961158
iter 240 value 27389.073013
iter 250 value 27367.434025
iter 260 value 27341.926388
iter 270 value 27319.858272
iter 280 value 27299.788159
iter 290 value 27282.338049
iter 300 value 27265.426207
iter 310 value 27245.670922
iter 320 value 27224.415669
iter 330 value 27195.444176
iter 340 value 27166.201807
iter 350 value 2714

In [55]:
Y_hat_ts<- predict(model,X_ts)
Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))

Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
Y_hat <- ifelse(Y_hat[,1] > threshold,"functional",ifelse(Y_hat[,2] > threshold,"functional_needs_repair","non_functional"))
Y_new <- Y[idx_ts,]
acc <- confusionMatrix(as.factor(Y_hat),as.factor(Y_new))$overall['Accuracy']
acc

Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(Y_new)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


Accuracy 
0.7445286

# Model Selection

In [60]:
#cross validation of multiple models

CV_folds <- 5

size_CV <-floor(N/CV_folds)

CV_err<-matrix(0,nrow=1,ncol=CV_folds)
threshold <- 0.5

net <- 0
part <- 0
randfor <- 0


for (i in 1:CV_folds) {
    
    idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
    X_ts<-X_final[idx_ts,]  
    Y_ts<-Y[idx_ts,]
     
    idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
    X_tr<-X_final[idx_tr,]
    Y_tr<-Y[idx_tr,]         
        # Create a dataset including only the first nb_components principal components
    DS<-cbind(X_tr,status_group=Y_tr)
    DS$status_group<-factor(DS$status_group)

    # Model fit (using lm function)
    model<- nnet(status_group~.,data=DS, size=5, decay=1.0e-5, maxit=1000, MaxNWts=2000,trace=FALSE)#1000 max iterations ?

    # Model predict
    Y_hat_ts<- predict(model,X_ts)
    Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))
                 
    Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
    Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
    Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
    Y_hat <- ifelse(Y_hat[,1] > threshold,"functional",ifelse(Y_hat[,2] > threshold,"functional_needs_repair","non_functional"))
    acc_nnet <- confusionMatrix(as.factor(Y_hat),as.factor(Y_ts))$overall['Accuracy']
    print(acc_nnet)
         
    #rpart
    tree<- rpart(status_group~.,data=DS, method='class')

    # Model predict
    Y_hat_ts<- predict(tree,X_ts)
    Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))
                 
    Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
    Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
    Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
    Y_hat <- ifelse(Y_hat[,1] > threshold,"functional",ifelse(Y_hat[,2] > threshold,"functional_needs_repair","non_functional"))
    acc_rpart <- confusionMatrix(as.factor(Y_hat),as.factor(Y_ts))$overall['Accuracy']
    print(acc_rpart)
                     
    #randomForest
    forest<- randomForest(status_group~.,data=DS)

    # Model predict
    Y_hat_ts<- predict(forest,X_ts)
    Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))
                 
    Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
    Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
    Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
    Y_hat <- ifelse(Y_hat[,1] > threshold,"functional",ifelse(Y_hat[,2] > threshold,"functional_needs_repair","non_functional"))
    acc_forest <- confusionMatrix(as.factor(Y_hat),as.factor(Y_ts))$overall['Accuracy']
    print(acc_forest)
                     
    a<-c(acc_nnet, acc_rpart, acc_forest)
                     
    ind <- which(a==max(a))
    if (ind ==1){
        net <- net+1
    }
    else if (ind ==2){
        part <- part + 1
    }
    else{
        randfor <- randfor + 1
    }
    print(net)
    print(part)
    print(randfor)
} 

# weights:  1633
initial  value 80112.960156 
iter  10 value 41879.357067
iter  20 value 41519.898024
iter  30 value 41357.272670
iter  40 value 41276.530748
iter  50 value 41257.972774
iter  60 value 41241.592502
iter  70 value 41237.299398
iter  80 value 41221.724799
iter  90 value 41185.421706
iter 100 value 41172.148516
iter 110 value 41049.570957
iter 120 value 41030.616762
iter 130 value 40974.113933
iter 140 value 40685.925192
iter 150 value 40025.675380
iter 160 value 38962.138082
iter 170 value 38189.477240
iter 180 value 37958.220321
iter 190 value 37273.820993
iter 200 value 35376.732386
iter 210 value 34434.356527
iter 220 value 33977.524761
iter 230 value 33496.518142
iter 240 value 33228.839868
iter 250 value 33075.434243
iter 260 value 32890.838876
iter 270 value 32749.601528
iter 280 value 32721.132290
iter 290 value 32714.791551
iter 300 value 32712.309564
iter 310 value 32710.144631
iter 320 value 32707.702158
iter 330 value 32706.740619
iter 340 value 32705.379669
it

Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(Y_ts)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


 Accuracy 
0.7128788 


Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(Y_ts)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


 Accuracy 
0.7080808 


...

 Accuracy 
0.7128788 


Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(Y_ts)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


 Accuracy 
0.7128788 


Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(Y_ts)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


 Accuracy 
0.7128788 
[1] 0


# XGBoost

[1] 3

# One hot to be tested and compared

In [23]:
train_idx <- 0:40000
test_idx <- 40001:59400

train_data <- X_final[train_idx,]
test_data <- X_final[test_idx,]

train_label <- (Y[train_idx,]) #wont work if did "data.frame"
test_label <-  (Y[test_idx,])

In [45]:
library(nnet)


model_one_hot<-nnet(x=train_data,
               y=class.ind(as.factor(train_label)),
               size=5,
               skip=FALSE,
               trace=T, 
               maxit=1000)
model_one_hot

Y_pred<-predict(model_one_hot,test_data)

threshold<-0.5
nrow(test_label)
Y_hat_one_hot <- t(apply(Y_pred, 1, function(x)(x-min(x))/(max(x)-min(x))))
#Y_hat_one_hot <- as.data.frame(matrix(0, ncol = 3, nrow = nrow(test_label)))

Y_hat_one_hot[,1] <- ifelse(Y_hat_one_hot[,1] < 0.99,0,1)
Y_hat_one_hot[,2] <- ifelse(Y_hat_one_hot[,2] < 0.99,0,1)
Y_hat_one_hot[,3] <- ifelse(Y_hat_one_hot[,3] < 0.99,0,1)

# weights:  1633
initial  value 26078.227402 
iter  10 value 21895.669696
iter  20 value 21802.383463
iter  30 value 21676.995487
iter  40 value 21660.506507
iter  50 value 21637.248870
iter  60 value 21609.396409
iter  70 value 21585.696625
iter  80 value 21573.301902
iter  90 value 21564.546076
iter 100 value 21558.271204
iter 110 value 21549.913068
iter 120 value 21545.496012
iter 130 value 21515.398955
iter 140 value 21459.271393
iter 150 value 21394.389823
iter 160 value 21239.946088
iter 170 value 21169.757758
iter 180 value 20964.721279
iter 190 value 20527.124532
iter 200 value 18967.956998
iter 210 value 18409.958517
iter 220 value 17571.413096
iter 230 value 16601.147282
iter 240 value 16483.469577
iter 250 value 16343.567363
iter 260 value 16135.240763
iter 270 value 15785.726501
iter 280 value 15469.761664
iter 290 value 15372.663686
iter 300 value 15259.437134
iter 310 value 15129.693057
iter 320 value 14986.913431
iter 330 value 14867.105315
iter 340 value 14804.074347
it

a 159-10-3 network with 1633 weights
options were -

NULL

In [46]:
Y_hat_one_hot <- ifelse(Y_hat_one_hot[,1] > threshold,"functional",ifelse(Y_hat_one_hot[,2] > threshold,"functional_needs_repair","non_functional"))
Y_hat_one_hot

40001            40002            40003            40004 
"non_functional"     "functional"     "functional" "non_functional" 
           40005            40006            40007            40008 
    "functional"     "functional"     "functional"     "functional" 
           40009            40010            40011            40012 
    "functional"     "functional" "non_functional" "non_functional" 
           40013            40014            40015            40016 
    "functional"     "functional" "non_functional"     "functional" 
           40017            40018            40019            40020 
    "functional"     "functional"     "functional"     "functional" 
           40021            40022            40023            40024 
"non_functional" "non_functional"     "functional"     "functional" 
           40025            40026            40027            40028 
    "functional"     "functional" "non_functional"     "functional" 
           40029            40030            40031            40032 
    "functional" "non_functional"     "functional"     "functional" 
           40033            40034            40035            40036 
    "functional"     "functional"     "functional"     "functional" 
           40037            40038            40039            40040 
"non_functional"     "functional"     "functional" "non_functional" 
           40041            40042            40043            40044 
    "functional"     "functional" "non_functional"     "functional" 
           40045            40046            40047            40048 
    "functional"     "functional"     "functional" "non_functional" 
           40049            40050            40051            40052 
    "functional" "non_functional" "non_functional" "non_functional" 
           40053            40054            40055            40056 
    "functional" "non_functional"     "functional"     "functional" 
           40057            40058            40059            40060 
    "functional"     "functional"     "functional"     "functional" 
           40061            40062            40063            40064 
    "functional"     "functional" "non_functional"     "functional" 
           40065            40066            40067            40068 
    "functional"     "functional"     "functional"     "functional" 
           40069            40070            40071            40072 
"non_functional"     "functional" "non_functional"     "functional" 
           40073            40074            40075            40076 
    "functional"     "functional" "non_functional" "non_functional" 
           40077            40078            40079            40080 
"non_functional"     "functional"     "functional" "non_functional" 
           40081            40082            40083            40084 
"non_functional"     "functional" "non_functional"     "functional" 
           40085            40086            40087            40088 
    "functional" "non_functional" "non_functional"     "functional" 
           40089            40090            40091            40092 
    "functional"     "functional"     "functional"     "functional" 
           40093            40094            40095            40096 
    "functional"     "functional" "non_functional"     "functional" 
           40097            40098            40099            40100 
    "functional"     "functional"     "functional"     "functional" 
           40101            40102            40103            40104 
    "functional"     "functional" "non_functional"     "functional" 
           40105            40106            40107            40108 
"non_functional"     "functional"     "functional"     "functional" 
           40109            40110            40111            40112 
    "functional" "non_functional"     "functional"     "functional" 
           40113            40114            40115            40116 
    "functional"     "functional"     "functional"     "functional" 
         

In [47]:
confusionMatrix(as.factor(Y_hat_one_hot),as.factor(test_label))

Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat_one_hot), as.factor(test_label)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


Confusion Matrix and Statistics

                         Reference
Prediction                functional functional_needs_repair non_functional
  functional                    9497                    1047           2534
  functional_needs_repair          0                       0              0
  non_functional                1128                     323           4871

Overall Statistics
                                          
               Accuracy : 0.7406          
                 95% CI : (0.7344, 0.7468)
    No Information Rate : 0.5477          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4878          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: functional Class: functional_needs_repair
Sensitivity                     0.8938                        0.00000
Specificity                     0.5919                    

In [49]:
saveRDS(model_one_hot, "model_one_hot.rds")

In [24]:
my_model <- readRDS("model_one_hot.rds")
Y_pred<-predict(my_model,test_data)

In [25]:
Y_pred

,functional,functional_needs_repair,non_functional
40001,0.2862838,0.26119212,0.40544007
40002,0.4921563,0.05589336,0.41623964
40003,0.6418489,0.08390003,0.26888278
40004,0.1705083,0.03890392,0.78369816
40005,0.8959849,0.08357572,0.05484649
40006,0.7829691,0.07554163,0.15236481
40007,0.7787799,0.07512350,0.15570922
40008,0.9025966,0.08495731,0.05101057
40009,0.9141840,0.08765247,0.04438250
40010,0.6997207,0.08093488,0.18335137


In [34]:
Y_hat <- t(apply(Y_pred, 1, function(x)(x-min(x))/(max(x)-min(x))))
Y_hat

,functional,functional_needs_repair,non_functional
40001,0.17394838,0.0000000000,1.00000000
40002,1.00000000,0.0000000000,0.82598417
40003,1.00000000,0.0000000000,0.33154072
40004,0.17669893,0.0000000000,1.00000000
40005,1.00000000,0.0341551775,0.00000000
40006,1.00000000,0.0000000000,0.10859514
40007,1.00000000,0.0000000000,0.11452425
40008,1.00000000,0.0398629606,0.00000000
40009,1.00000000,0.0497469555,0.00000000
40010,1.00000000,0.0000000000,0.16551201


In [30]:
                 
threshold <- 0.5
                 
Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
Y_hat <- ifelse(Y_hat[,1] > threshold,"functional",ifelse(Y_hat[,2] > threshold,"functional_needs_repair","non_functional"))
cm <- confusionMatrix(as.factor(Y_hat),as.factor(test_label))

Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(test_label)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


In [33]:
cm <- str(cm)
cm$table

                         Reference
Prediction                functional functional_needs_repair non_functional
  functional                    9497                    1047           2534
  functional_needs_repair          0                       0              0
  non_functional                1128                     323           4871

In [ ]:
ROC_AUC <- function(pred, init){#On peut ecarter car prediction pas terrible pour ce genre de probleme
    cm <- confusionMatrix(as.factor(Y_hat),as.factor(test_label))
    cm <- str(cm)
    cm$table[1,1]
    
    FP <- cm$table[2,1]
    TP <- cm$table[2,2]
    N_N <- sum(cm$table[,1]) # Total number of nonspam
    N_P <- sum(cm$table[,2]) # Total number of spam

    FPR <- c(FPR,FP/N_N)
    TPR <- c(TPR,TP/N_P)
}

In [ ]:
data.frame(model$variable.importance)# for rpart

In [ ]:
#install.packages("rpart.plot")
prp(model)
rpart.plot(model)

In [ ]:
# plot tree
plot(model, uniform=TRUE,
   main="Classification Tree - Rpart")
text(model, use.n=TRUE, all=TRUE, cex=.8)

# Data preprocessing

## Missing value imputation

## Feature engineering

## Feature selection



# Model selection

## Model 1

## Model 2

## Model 3

#### Example of simple equation
\begin{equation}
e = mc^2
\end{equation}

#### Example of matrix equation - Cross product formula:

\begin{equation*}
\mathbf{V}_1 \times \mathbf{V}_2 =  \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
\frac{\partial X}{\partial u} &  \frac{\partial Y}{\partial u} & 0 \\
\frac{\partial X}{\partial v} &  \frac{\partial Y}{\partial v} & 0
\end{vmatrix}
\end{equation*}

#### Example of multiline equation - The Lorenz Equations:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

#### Example of Markdown Table:

| This | is   |
|------|------|
|   a  | table|


# Alternative models





# Conclusions